In [3]:
# DL for Microscopy
#cell counting in deepchem

In [4]:
##setup tensorflow v1
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
!wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
!chmod +x Anaconda3-2019.10-Linux-x86_64.sh
!bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2020-07-26 14:14:19--  https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530308481 (506M) [application/x-sh]
Saving to: ‘Anaconda3-2019.10-Linux-x86_64.sh’

Anaconda3-2019.10-L 100%[===================>] 505.74M  93.3MB/s    in 5.2s    

2020-07-26 14:14:24 (96.9 MB/s) - ‘Anaconda3-2019.10-Linux-x86_64.sh’ saved [530308481/530308481]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - anaconda-navigator==1.9.7=py3

In [6]:
import deepchem as dc
dc.__version__ #should match with the installed

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



'2.3.0'

In [7]:
#download image data
!wget https://data.broadinstitute.org/bbbc/BBBC005/BBBC005_v1_images.zip

--2020-07-26 14:37:21--  https://data.broadinstitute.org/bbbc/BBBC005/BBBC005_v1_images.zip
Resolving data.broadinstitute.org (data.broadinstitute.org)... 69.173.92.29
Connecting to data.broadinstitute.org (data.broadinstitute.org)|69.173.92.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1882973059 (1.8G) [application/zip]
Saving to: ‘BBBC005_v1_images.zip’

BBBC005_v1_images.z 100%[===================>]   1.75G  1.42MB/s    in 21m 43s 

2020-07-26 14:59:05 (1.38 MB/s) - ‘BBBC005_v1_images.zip’ saved [1882973059/1882973059]



In [8]:
!unzip BBBC005_v1_images.zip

Streaming output truncated to the last 5000 lines.
  inflating: BBBC005_v1_images/SIMCEPImages_E18_C74_F14_s15_w2.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_P12_C48_F48_s05_w2.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_K15_C61_F32_s01_w1.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_L12_C48_F35_s12_w2.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_N09_C35_F42_s18_w1.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_I08_C31_F26_s07_w2.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_M18_C74_F39_s13_w2.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_E21_C87_F14_s14_w1.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_A01_C1_F1_s25_w2.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_J15_C61_F29_s23_w1.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_N02_C5_F42_s07_w2.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_I11_C44_F26_s15_w1.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_A16_C66_F1_s19_w2.TIF  
  inflating: BBBC005_v1_images/SIMCEPImages_C21_C87_F7_s

In [10]:
import numpy as np
import os
import re

In [11]:
image_dir = 'BBBC005_v1_images'
files = []
labels = []
for f in os.listdir(image_dir):
  if f.endswith('.TIF'):
    files.append(os.path.join(image_dir, f))
    labels.append(int(re.findall('_C(.*?)_', f)[0]))
loader = dc.data.ImageLoader()
dataset = loader.featurize(files, np.array(labels))

In [17]:
import deepchem.models.tensorgraph.layers as layers

In [14]:
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset, seed=123)

In [44]:
del model

In [37]:
learning_rate = dc.models.optimizers.ExponentialDecay(0.001, 0.9,250)
#above line is different in book
#model = dc.models.TensorGraph(learning_rate=learning_rate, model_dir='model')
# for loading the pre trained model as mentioned in the book
model = dc.models.TensorGraph(learning_rate=learning_rate, model_dir='models/model')
features = layers.Feature(shape=(None, 520, 696))
labels = layers.Label(shape=(None,))

In [38]:
prev_layer = features
for num_outputs in [16, 32, 64, 128, 256]:
  prev_layer = layers.Conv2D(num_outputs, kernel_size=5, stride=2,in_layers=prev_layer)

output = layers.Dense(1, in_layers=layers.Flatten(prev_layer))

model.add_output(output)

loss = layers.ReduceSum(layers.L2Loss(in_layers=(output, labels)))

model.set_loss(loss)

In [20]:
#model training
#model.fit(train_dataset, nb_epoch=50)




Instructions for updating:
If using Keras pass *_constraint arguments to layers.









20.800748942071802

In [21]:
#testing
y_pred = model.predict(test_dataset).flatten()
print(np.sqrt(np.mean((y_pred-test_dataset.y)**2)))

2.7211521453040683


In [42]:
pwd

'/content'

In [73]:
#!wget https://s3-us-west-1.amazonaws.com/deepchem.io/featurized_datasets/microscopy_models.zip
#!unzip microscopy_models.zip

In [34]:
pwd

'/content/models'

In [74]:
del model

In [75]:
#load model, above procedure gives error so follow another idea
learning_rate = dc.models.optimizers.ExponentialDecay(0.001, 0.9, 250)
model = dc.models.TensorGraph(learning_rate=learning_rate, model_dir='model')
features = layers.Feature(shape=(None, 520, 696))
labels = layers.Label(shape=(None,))
prev_layer = features
for num_outputs in [16, 32, 64, 128, 256]:
  prev_layer = layers.Conv2D(num_outputs, kernel_size=5, stride=2, in_layers=prev_layer)
output = layers.Dense(1, in_layers=layers.Flatten(prev_layer))
model.add_output(output)
loss = layers.ReduceSum(layers.L2Loss(in_layers=(output, labels)))
model.set_loss(loss)

In [65]:
cd ../

/content


In [71]:
#!unzip microscopy_models.zip

In [80]:
model.get_checkpoints()

['model/model-3999', 'model/model-4999', 'model/model-5999', 'model/model-6999', 'model/model-7700']

In [ ]:
#model.load_from_pretrained()

In [83]:
## then restore a model
model.restore()

INFO:tensorflow:Restoring parameters from model/model-7700


NotFoundError: ignored

In [ ]:
#using pretrained model
# mkdir models
# cd models
# wget https://s3-us-west-1.amazonaws.com/deepchem.io/featurized_datasets/microscopy_models.zip
# unzip microscopy_models.zip
## then restore a model
#model.restore()

In [72]:
#predict on the model------ pre trained model did not load
# I trained from scratch it took 3 hours for 50 epochs
#y_pred = model.predict(test_dataset).flatten()
#print(np.sqrt(np.mean((y_pred-test_dataset.y)**2)))

2.7211521453040683


In [86]:
y_pred[0:10]

array([27.126934 , 22.569012 , 43.501827 , 10.085748 , 69.83527  ,
       39.576748 , 18.093826 , 91.25696  , 82.43254  ,  5.2407064],
      dtype=float32)

In [ ]:
#another thing would be to load pretrained model, 
# train it little more and then predict
# model.fit(train_dataset, nb_epoch=50)
# y_pred = model.predict(test_dataset).flatten()
# print(np.sqrt(np.mean((y_pred-test_dataset.y)**2)))

In [82]:
# metric in the 3 cases :
# 1. Training from scratch (totalepochs = 50) and predict: 2.7211521453040683
# 2. Load pretrained (total epochs = x maybe 50) and predict: NA
# 3. Load and then retrain (total epochs = x+50 maybe (50+50)) and predict: NA

In [84]:
# Couldn't debug the error of loading the model provided by authors on github
# I can load the model I trained but not succesful with the model provided on github
# going forward with segmentation task

In [87]:
#!wget https://data.broadinstitute.org/bbbc/BBBC005/BBBC005_v1_ground_truth.zip

--2020-07-26 19:22:13--  https://data.broadinstitute.org/bbbc/BBBC005/BBBC005_v1_ground_truth.zip
Resolving data.broadinstitute.org (data.broadinstitute.org)... 69.173.92.29
Connecting to data.broadinstitute.org (data.broadinstitute.org)|69.173.92.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12158428 (12M) [application/zip]
Saving to: ‘BBBC005_v1_ground_truth.zip’

BBBC005_v1_ground_t 100%[===================>]  11.59M  1.40MB/s    in 8.7s    

2020-07-26 19:22:23 (1.33 MB/s) - ‘BBBC005_v1_ground_truth.zip’ saved [12158428/12158428]



In [89]:
#!unzip BBBC005_v1_ground_truth.zip

In [90]:
image_dir = 'BBBC005_v1_images'
label_dir = 'BBBC005_v1_ground_truth'
rows = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L','M', 'N', 'O', 'P')
blurs = (1, 4, 7, 10, 14, 17, 20, 23, 26, 29, 32, 35, 39, 42, 45, 48)
files = []
labels = []
for f in os.listdir(label_dir):
  if f.endswith('.TIF'):
    for row, blur in zip(rows, blurs):
      fname = f.replace('_F1', '_F%d'%blur).replace('_A', '_%s'%row)
      files.append(os.path.join(image_dir, fname))
      labels.append(os.path.join(label_dir, f))
      
loader = dc.data.ImageLoader()
dataset = loader.featurize(files, labels)

In [91]:
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset, seed=123)

In [92]:
#Unet architecture

In [98]:
del model

In [104]:
learning_rate = dc.models.optimizers.ExponentialDecay(0.01, 0.9, 250)
model = dc.models.TensorGraph(learning_rate=learning_rate, model_dir='segmentation_old')
features = layers.Feature(shape=(None, 520, 696, 1)) / 255.0
labels = layers.Label(shape=(None, 520, 696, 1)) / 255.0
# Downsample three times.
conv1 = layers.Conv2D(16, kernel_size=5, stride=2, in_layers=features)
conv2 = layers.Conv2D(32, kernel_size=5, stride=2, in_layers=conv1)
conv3 = layers.Conv2D(64, kernel_size=5, stride=2, in_layers=conv2)
# Do a 1x1 convolution.
conv4 = layers.Conv2D(64, kernel_size=1, stride=1, in_layers=conv3)
# Upsample three times.
concat1 = layers.Concat(in_layers=[conv3, conv4], axis=3)
deconv1 = layers.Conv2DTranspose(32, kernel_size=5, stride=2, in_layers=concat1)
concat2 = layers.Concat(in_layers=[conv2, deconv1], axis=3)
deconv2 = layers.Conv2DTranspose(16, kernel_size=5, stride=2, in_layers=concat2)
concat3 = layers.Concat(in_layers=[conv1, deconv2], axis=3)
deconv3 = layers.Conv2DTranspose(1, kernel_size=5, stride=2, in_layers=concat3)
# Compute the final output.
concat4 = layers.Concat(in_layers=[features, deconv3], axis=3)
logits = layers.Conv2D(1, kernel_size=5, stride=1, activation_fn=None, in_layers=concat4)
output = layers.Sigmoid(logits)
model.add_output(output)
loss = layers.ReduceSum(layers.SigmoidCrossEntropy(in_layers=(labels, logits)))
model.set_loss(loss)

In [100]:
pwd

'/content'

In [105]:
#model.restore()

In [102]:
# same error coming in loading pretrained model
# skipping this chapter lots of issues

In [103]:
# training Unet from scratch for 5 epochs just to see how it works

In [106]:
model.fit(train_dataset, nb_epoch=5, checkpoint_interval=100)

1035992.4084507042

In [107]:
scores = []
for x, y, w, id in test_dataset.itersamples():
  y_pred = model.predict_on_batch([x]).squeeze()
  scores.append(np.mean((y>0) == (y_pred>0.5)))
print(np.mean(scores))

0.9882743092971744


In [109]:
# after 5 epochs from scratch 0.9882 ~ 98.82%
# as per book after 50 epochs ~ 0.9899 98.99%

In [108]:
#chapter over but unable to load pretrained models, will come back to this 